In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from statsmodels.tsa.arima.model import ARIMA

In [52]:
def fetch_historical_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data['Adj Close'].values.reshape(-1, 1)

def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data)-1:
            break
        seq_x, seq_y = data[i:end_ix], data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def create_lstm_model(n_steps, n_features):
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(units=1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

def predict_future_prices(model, X):
    return model.predict(X)

def calculate_expected_return(predicted_prices, current_price):
    expected_returns = (predicted_prices - current_price) / current_price
    return expected_returns

def monte_carlo_simulation(predicted_prices, num_simulations):
    returns = []
    for _ in range(num_simulations):
        simulated_prices = np.random.normal(predicted_prices, predicted_prices * 0.1) # Assume 10% volatility
        returns.append((simulated_prices[-1] - predicted_prices[-1]) / predicted_prices[-1])
    return np.mean(returns)

def predict_best_future(symbol, start_date, end_date, contract_lengths):
    historical_data = fetch_historical_data(symbol, start_date, end_date)

    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(historical_data)

    n_steps = 10
    X, y = prepare_data(scaled_data, n_steps)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    n_features = 1
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

    model = create_lstm_model(n_steps, n_features)
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    future_prices = []
    for contract_length in contract_lengths:
        future_dates = pd.date_range(end_date, periods=contract_length)
        future_data = fetch_historical_data(symbol, end_date, future_dates[-1])
        scaled_future_data = scaler.transform(future_data)
        X_future, _ = prepare_data(scaled_future_data, n_steps)
        X_future = X_future.reshape((X_future.shape[0], X_future.shape[1], n_features))
        predicted_prices = predict_future_prices(model, X_future)
        future_prices.append(predicted_prices[-1][0])

    current_price = historical_data[-1][0]
    expected_returns = []
    for future_price in future_prices:
        expected_returns.append(calculate_expected_return(np.array([future_price]), current_price)[0])

    num_simulations = 1000
    monte_carlo_returns = []
    for future_price in future_prices:
        monte_carlo_returns.append(monte_carlo_simulation(np.array([future_price]), num_simulations))

    best_contract_index = np.argmax(monte_carlo_returns)
    best_contract_length = contract_lengths[best_contract_index]
    best_expected_return = monte_carlo_returns[best_contract_index]

    return best_contract_length, best_expected_return

ticker_list = ["^NDX", "AAPL", "AMZN","MSFT", "GOOGL","TSLA", "NFLX", "JPM", "NVDA", "V"]
results_list = list()
for symbol in ticker_list:
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)
    contract_lengths = [30, 60, 90] # Contract lengths to consider
    best_contract_length, best_expected_return = predict_best_future(symbol, start_date, end_date, contract_lengths)
    print(f"The contract with length {best_contract_length} days has the best expected return of {best_expected_return:.4f}.")
    results_list.append((symbol, best_contract_length, best_expected_return))
max_value = max(results_list, key=lambda x: x[2])
max_value

[*********************100%%**********************]  1 of 1 completed
C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%%**********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step


[*********************100%%**********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step


[*********************100%%**********************]  1 of 1 completed

...

[*********************100%%**********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step


[*********************100%%**********************]  1 of 1 completed

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
The contract with length 60 days has the best expected return of 0.0035.


('MSFT', 30, 0.008554557033570241)

In [18]:
def fetch_historical_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data['Adj Close'].values

def create_arima_model(data, order):
    model = ARIMA(data, order=order)
    fitted_model = model.fit()
    return fitted_model

def predict_future_prices(model, steps):
    forecast = model.forecast(steps=steps)
    return forecast

def monte_carlo_simulation(predicted_prices, num_simulations):
    returns = []
    for _ in range(num_simulations):
        simulated_prices = np.random.normal(predicted_prices, predicted_prices * 0.1) # Assume 10% volatility
        returns.append((simulated_prices[-1] - predicted_prices[-1]) / predicted_prices[-1])
    return np.mean(returns)

def predict_best_future(symbol, start_date, end_date, contract_lengths, arima_order):
    historical_data = fetch_historical_data(symbol, start_date, end_date)

    model = create_arima_model(historical_data, arima_order)

    future_prices = []
    for contract_length in contract_lengths:
        predicted_prices = predict_future_prices(model, steps=contract_length)
        future_prices.append(predicted_prices[-1])

    num_simulations = 1000
    monte_carlo_returns = []
    for future_price in future_prices:
        monte_carlo_returns.append(monte_carlo_simulation(np.array([future_price]), num_simulations))

    best_contract_index = np.argmax(monte_carlo_returns)
    best_contract_length = contract_lengths[best_contract_index]
    best_expected_return = monte_carlo_returns[best_contract_index]

    return best_contract_length, best_expected_return

ticker_list = ["^NDX", "AAPL", "AMZN","MSFT", "GOOGL","TSLA", "NFLX", "JPM", "NVDA", "V"]
results_list = list()
for symbol in ticker_list:
    start_date = datetime(2024, 5, 12)
    end_date = datetime(2025, 12, 31)
    contract_lengths = [30, 60, 90] # Contract lengths to consider
    arima_order = (5, 1, 0) # ARIMA order (p, d, q)
    best_contract_length, best_expected_return = predict_best_future(symbol, start_date, end_date, contract_lengths, arima_order)
    print(f"The contract with length {best_contract_length} days has the best expected return of {best_expected_return:.4f}.")
    results_list.append((symbol, best_contract_length, best_expected_return))
max_value = max(results_list, key=lambda x: x[2])
max_value

[*********************100%%**********************]  1 of 1 completed
C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
[*********************100%%**********************]  1 of 1 completed
C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to

The contract with length 30 days has the best expected return of 0.0021.


C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
[*********************100%%**********************]  1 of 1 completed
C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


The contract with length 30 days has the best expected return of 0.0010.


[*********************100%%**********************]  1 of 1 completed

...

The contract with length 60 days has the best expected return of 0.0033.



C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


The contract with length 60 days has the best expected return of 0.0079.


C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


('V', 60, 0.007891645971454477)

In [19]:
def fetch_historical_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data['Adj Close'].values

def create_arima_model(data, order):
    model = ARIMA(data, order=order)
    fitted_model = model.fit()
    return fitted_model

def predict_future_prices(model, steps):
    forecast = model.forecast(steps=steps)
    return forecast

def monte_carlo_simulation(predicted_prices, num_simulations):
    returns = []
    for _ in range(num_simulations):
        simulated_prices = np.random.normal(predicted_prices, predicted_prices * 0.1) # Assume 10% volatility
        returns.append((simulated_prices[-1] - predicted_prices[-1]) / predicted_prices[-1])
    return np.mean(returns)

def predict_best_forward(symbol, start_date, end_date, forward_lengths, arima_order):
    historical_data = fetch_historical_data(symbol, start_date, end_date)

    model = create_arima_model(historical_data, arima_order)

    future_prices = []
    for forward_length in forward_lengths:
        predicted_prices = predict_future_prices(model, steps=forward_length)
        future_prices.append(predicted_prices[-1])

    num_simulations = 1000
    monte_carlo_returns = []
    for future_price in future_prices:
        monte_carlo_returns.append(monte_carlo_simulation(np.array([future_price]), num_simulations))

    best_contract_index = np.argmax(monte_carlo_returns)
    best_contract_length = forward_lengths[best_contract_index]
    best_expected_return = monte_carlo_returns[best_contract_index]

    return best_contract_length, best_expected_return

ticker_list = ["^NDX", "AAPL", "AMZN","MSFT", "GOOGL","TSLA", "NFLX", "JPM", "NVDA", "V"]
results_list = list()
for symbol in ticker_list:
    start_date = datetime(2024, 5, 12)
    end_date = datetime(2025, 12, 31)
    forward_lengths = [30, 60, 90] # Contract lengths to consider
    arima_order = (5, 1, 0) # ARIMA order (p, d, q)
    best_contract_length, best_expected_return = predict_best_forward(symbol, start_date, end_date, forward_lengths, arima_order)
    print(f"The contract with length {best_contract_length} days has the best expected return of {best_expected_return:.4f}.")
    results_list.append((symbol, best_contract_length, best_expected_return))
max_value = max(results_list, key=lambda x: x[2])
max_value

[*********************100%%**********************]  1 of 1 completed
C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
[*********************100%%**********************]  1 of 1 completed


The contract with length 60 days has the best expected return of 0.0041.


C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
[*********************100%%**********************]  1 of 1 completed
C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


The contract with length 90 days has the best expected return of 0.0006.


[*********************100%%**********************]  1 of 1 completed

...

The contract with length 30 days has the best expected return of 0.0018.



C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


The contract with length 60 days has the best expected return of 0.0029.


C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


('NFLX', 90, 0.00587746486440832)

In [54]:
def fetch_historical_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data['Adj Close'].values.reshape(-1, 1)

def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data)-1:
            break
        seq_x, seq_y = data[i:end_ix], data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def create_lstm_model(n_steps, n_features):
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(units=1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

def predict_future_prices(model, X):
    return model.predict(X)

def calculate_expected_return(predicted_prices, current_price):
    expected_returns = (predicted_prices - current_price) / current_price
    return expected_returns

def monte_carlo_simulation(predicted_prices, num_simulations):
    returns = []
    for _ in range(num_simulations):
        simulated_prices = np.random.normal(predicted_prices, predicted_prices * 0.1) # Assume 10% volatility
        returns.append((simulated_prices[-1] - predicted_prices[-1]) / predicted_prices[-1])
    return np.mean(returns)

def predict_best_forward(symbol, start_date, end_date, forward_lengths):
    historical_data = fetch_historical_data(symbol, start_date, end_date)

    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(historical_data)

    n_steps = 10
    X, y = prepare_data(scaled_data, n_steps)

    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    n_features = 1
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

    model = create_lstm_model(n_steps, n_features)
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    future_prices = []
    for forward_length in forward_lengths:
        future_dates = pd.date_range(end_date, periods=forward_length)
        future_data = fetch_historical_data(symbol, end_date, future_dates[-1])
        scaled_future_data = scaler.transform(future_data)
        X_future, _ = prepare_data(scaled_future_data, n_steps)
        X_future = X_future.reshape((X_future.shape[0], X_future.shape[1], n_features))
        predicted_prices = predict_future_prices(model, X_future)
        future_prices.append(predicted_prices[-1][0])

    current_price = historical_data[-1][0]
    expected_returns = []
    for future_price in future_prices:
        expected_returns.append(calculate_expected_return(np.array([future_price]), current_price)[0])

    num_simulations = 1000
    monte_carlo_returns = []
    for future_price in future_prices:
        monte_carlo_returns.append(monte_carlo_simulation(np.array([future_price]), num_simulations))

    best_contract_index = np.argmax(monte_carlo_returns)
    best_contract_length = forward_lengths[best_contract_index]
    best_expected_return = monte_carlo_returns[best_contract_index]

    return best_contract_length, best_expected_return


ticker_list = ["^NDX", "AAPL", "AMZN","MSFT", "GOOGL","TSLA", "NFLX", "JPM", "NVDA", "V"]
results_list = list()
for symbol in ticker_list:
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)
    forward_lengths = [30, 60, 90] # Contract lengths to consider
    best_contract_length, best_expected_return = predict_best_forward(symbol, start_date, end_date, forward_lengths)
    print(f"The contract with length {best_contract_length} days has the best expected return of {best_expected_return:.4f}.")
    results_list.append((symbol, best_contract_length, best_expected_return))
max_value = max(results_list, key=lambda x: x[2])
max_value

[*********************100%%**********************]  1 of 1 completed
C:\Users\a-spyrosploussiou\AppData\Local\anaconda3_\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%%**********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step


[*********************100%%**********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step


[*********************100%%**********************]  1 of 1 completed

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step


[*********************100%%**********************]  1 of 1 completed

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


The contract with length 30 days has the best expected return of 0.0030.


('MSFT', 30, 0.003518580063006714)